<center><h1> Projeto de Machine Learning </h1><center>
<center><h1>Previsão de Chuvas na Austrália</h1><center>

------------------------------------------------
<center>Eduardo Selber, Henrique Badin e Luca Caruso<center>

## Preparando o ambiente

In [ ]:
#Importando as bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#import sweetviz as sv
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, root_mean_squared_error)
from sklearn.preprocessing import StandardScaler

#Carregando o dataset
df = pd.read_csv("..\data\weatherAUS.csv")
df.head(5)

In [ ]:
#Gerando um relatorio de analise exploratoria
#report = sv.analyze(df)
#report.show_notebook()

## Data Wrangling
Primeiro vamos formatar os dados da base de dados para prepará-lo para o modelo preditivo

In [ ]:
#Tratamento de valores ausentes: Deletando as linhas com os valores ausentes das colunas "Raintoday" e "RainTomorrow"
df = df.dropna(subset=["RainToday", "RainTomorrow"])

#Deixando apenas as colunas Date, Location, MinTemp, MaxTemp, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, Temp9am, Temp3pm, RainToday, RainTomorrow
df = df[[
    "Date",
    "Location",
    "MinTemp",
    "MaxTemp",
    "Humidity9am",
    "Humidity3pm",
    "Pressure9am",
    "Pressure3pm",
    "Temp9am",
    "Temp3pm",
    "RainToday",
    "RainTomorrow",
    "Rainfall",
    "WindGustSpeed",
    "WindSpeed9am",
    "WindSpeed3pm",
]]
df = df.dropna()

# #Tratamento dos outliers e features engineering:
#MinTemp
df = df[(df["MinTemp"] >= -10) & (df["MinTemp"] <= 30)]

#MaxTemp
df = df[(df["MaxTemp"] >= 5) & (df["MaxTemp"] <= 40)]

#Humidity9am não possui outliers
#Humidity3pm não possui outliers

#Pressure9am
df = df[(df["Pressure9am"] >= 1000) & (df["Pressure9am"] <= 1035)]

#Pressure3pm
df = df[(df["Pressure3pm"] >= 1000) & (df["Pressure3pm"] <= 1035)]

#Temp9am
df = df[(df["Temp9am"] >= 5) & (df["Temp9am"] <= 30)]

#Temp3pm
df = df[(df["Temp3pm"] >= 8) & (df["Temp3pm"] <= 35)]

#df =df[(df["Location"] =="Sydney")]

#Tratamento de variaveis categoricas
df["RainToday"] = df["RainToday"].map({"Yes": 1, "No": 0})
df['RainTomorrow'] = df['RainTomorrow'].replace({'Yes': 1, 'No': 0})

## EDA - Exploratory Data Analysis
Primeiro vamos analisar os dados que são fornecidos pela base de dados

### Colunas do dataset
- **Date** - A data da observação (formato: yyyy-mm-dd)
- **Location** - A localização da estação meteorológica (Nome da cidade - string)
- **MinTemp** - A temperatura mínima em graus Celsius (float)
- **MaxTemp** - A temperatura máxima em graus Celsius  (float)
- **Rainfall** - A quantidade de chuva medida em mm (float)
- **Evaporation** - A taxa de evaporação medida em mm (float)
- **Sunshine** - O número de horas de sol (float)
- **WindGustDir** - A direção da rajada de vento mais forte (string)
- **WindGustSpeed** - A velocidade (em km/h) da rajada de vento mais forte (float)
- **WindDir9am** - A direção do vento às 9am (string)
- **WindDir3pm** - A direção do vento às 3pm (string)
- **WindSpeed9am** - A velocidade do vento às 9am (em km/h) (float)
- **WindSpeed3pm** - A velocidade do vento às 3pm (em km/h) (float)
- **Humidity9am** - A umidade relativa às 9am (em %) (float)
- **Humidity3pm** - A umidade relativa às 3pm (em %) (float)
- **Pressure9am** - A pressão atmosférica reduzida ao nível do mar às 9am (em hpa) (float)
- **Pressure3pm** - A pressão atmosférica reduzida ao nível do mar às 3pm (em hpa) (float)
- **Cloud9am** - A fração de cobertura de nuvens às 9am (em oitavos) (float)
- **Cloud3pm** - A fração de cobertura de nuvens às 3pm (em oitavos) (float)
- **Temp9am** - A temperatura às 9am em graus Celsius (float)
- **Temp3pm** - A temperatura às 3pm em graus Celsius (float)
- **RainToday** - Se choveu (precipitação acima de 1mm) ou não (string)
- **RainTomorrow** - A variável alvo. Se choverá ou não amanhã (string)


In [ ]:
# Verificando as métricas do dataset
df.describe().transpose()

In [ ]:
df.shape


In [ ]:
df.head()

In [ ]:
locations_unicos = df['Location'].unique()
print(locations_unicos)

In [ ]:
df.columns

In [ ]:
target = df["RainTomorrow"]
target.shape

In [ ]:
target = target.dropna()
target.unique()

In [ ]:
#features = df[['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
#              'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
#             'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
#            'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
#           'Temp3pm', 'RainToday']]
features = df[[
    'Location',
    'Date',
    'MinTemp',
    'MaxTemp',
    'Humidity9am',
    'Humidity3pm',
    'Pressure9am',
    'Pressure3pm',
    'Temp9am',
    'Temp3pm',
    'RainToday',
]]
features.shape

In [ ]:
features.dtypes

In [ ]:
# Crie uma cópia do DataFrame original para evitar o SettingWithCopyWarning
features_c = features.copy()

# Converta a coluna Date para datetime
features_c['Date'] = pd.to_datetime(features_c['Date'], errors='coerce')

# Verifique se houve algum NaT após a conversão
if features_c['Date'].isnull().any():
    print("Existem valores não convertidos em 'Date':")
    print(features_c[features_c['Date'].isnull()])  # Mostre as linhas com NaT

# Adicionando as novas colunas se a conversão for bem-sucedida
if not features_c['Date'].isnull().any():
    features_c['Year'] = features_c['Date'].dt.year
    features_c['Month'] = features_c['Date'].dt.month
    features_c['Day'] = features_c['Date'].dt.day
    features_c['DayOfWeek'] = features_c['Date'].dt.dayofweek

    # Remova a coluna Date se não for mais necessária
    features_c = features_c.drop(columns=['Date'])
else:
    print("Não foi possível adicionar as colunas porque existem valores NaT.")

In [ ]:
features_c["RainToday"].describe()

In [ ]:
features_c['RainToday'] = features_c['RainToday'].map({'Yes': 1, 'No': 0})


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Codificando o campo Location em valores inteiros

# Agora remova a coluna original
df = df.drop(columns=['Location'])
# Ordenando os dados por data
df = df.sort_values('Date')

# Separando o DataFrame em treino, teste e validação
n = len(df)
train_end = int(0.6 * n)
test_end = int(0.8 * n)

x_train = df.iloc[:train_end].drop(columns=['RainTomorrow', 'Date'])
y_train = df.iloc[:train_end]['RainTomorrow']

x_test = df.iloc[train_end:test_end].drop(columns=['RainTomorrow', 'Date'])
y_test = df.iloc[train_end:test_end]['RainTomorrow']

x_val = df.iloc[test_end:].drop(columns=['RainTomorrow', 'Date'])
y_val = df.iloc[test_end:]['RainTomorrow']

# Treinando o modelo de regressão logística
#from imblearn.over_sampling import SMOTE

#smote = SMOTE(random_state=42)
#_train, y_train = smote.fit_resample(x_train_balanced, y_train_balanced)

In [ ]:
pd.Series(y_val).value_counts(normalize=True)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

In [ ]:
x_train.head()

In [ ]:
model = LogisticRegression(max_iter=1000, tol=1e-5, solver='liblinear')
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)


In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt

# Contagem de corretos e incorretos
certo = 0
total = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.iloc[i]:
        certo += 1
    total += 1

errado = total - certo

# Dados para o gráfico
labels = ['Correto', 'Incorreto']
values = [certo, errado]

# Criando o gráfico de barras
plt.bar(labels, values, color=['green', 'red'])
plt.xlabel('Previsões')
plt.ylabel('Quantidade')
plt.title('Quantidade de Previsões Corretas e Incorretas')
plt.show()
print(f'Acurácia: {certo/total*100}%')

In [ ]:
y_val_pred = model.predict(x_val)


In [ ]:
#Calculando a acurácia
accuracy_score(y_val, y_val_pred)

In [ ]:
# features_c = pd.get_dummies(
#     features_c,
#     columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'],
#     drop_first=True)

In [ ]:
features_c.dtypes

In [ ]:
target = target.map({'Yes': 1, 'No': 0})
target.value_counts()

In [ ]:
# Adicionando a variável target ao DataFrame de features
features_c['RainTomorrow'] = target

# Calculando a correlação apenas com a variável target
correlation_with_target = features_c.corr()['RainTomorrow'].drop(
    'RainTomorrow')

# Criando um DataFrame para o heatmap e ordenando pela correlação
correlation_df = correlation_with_target.to_frame().reset_index()
correlation_df.columns = ['Feature', 'Correlation']

# Ordenando pela correlação
correlation_df = correlation_df.sort_values(by='Correlation', ascending=False)

# Criando o heatmap sem números
plt.figure(figsize=(6, 6))  # Define o tamanho da figura
sns.heatmap(
    correlation_df.set_index('Feature'),
    annot=False,
    cmap='coolwarm',
    linewidths=.5,
    cbar_kws={"shrink": .8},
)

plt.title('Correlação das Features com RainTomorrow (Ordenado)')
plt.show()